# Data Reduction with AIJ

This tutorial will guide you through the process of reducing astronomical data from FITS files to create a normalized, phase-folded light curve of an eclipsing binary star system using AstroImageJ (AIJ). This is a fundamental skill in observational astronomy, particularly for studying variable stars.

## 1. Introduction

### What is an Eclipsing Binary?
An **eclipsing binary** is a system of two stars orbiting each other in such a way that, from our line of sight on Earth, one star periodically passes in front of the other. This passage causes a dip in the total observed brightness of the system. Studying these eclipses tells us a lot about the stars' properties, like their sizes, masses, and temperatures.

### What is a Light Curve?
A **light curve** is a graph that shows the brightness of an astronomical object over time. For an eclipsing binary, the light curve will show periodic dips in brightness corresponding to the eclipses. The shape and timing of these dips are crucial for analysis.

### What is AstroImageJ (AIJ)?
**AstroImageJ (AIJ)** is a powerful, free, Java-based image processing and analysis software specifically designed for astronomy. It's widely used for tasks like photometry (measuring brightness), image calibration, and creating light curves. You can download it from the official AIJ website: [https://www.astro.louisville.edu/software/astroimagej/](https://www.astro.louisville.edu/software/astroimagej/)

### Goal of this Tutorial
By the end of this tutorial, you will be able to:
* Load a series of FITS images into AIJ.
* Perform aperture photometry on your target eclipsing binary and suitable comparison stars.
* Generate a raw light curve.
* Normalize the light curve.
* **Phase fold the light curve using a known period and epoch.**
* Save your results.

### Prerequisites
* **AstroImageJ installed:** Ensure you have the latest version of AIJ installed on your computer.
* **Java Development Kit (JDK):** AIJ requires Java. Make sure you have a compatible JDK installed (usually OpenJDK or Oracle JDK).
* **FITS files:** You'll need a series of FITS (Flexible Image Transport System) files of an eclipsing binary. These are the raw data from your telescope observations.
* **Known Ephemeris (for phase folding):** To phase fold the light curve, you'll need:
    * The **orbital period (P)** of the binary system.
    * A **reference time of an event (T0 or Epoch)**, usually the time of a specific primary minimum (e.g., in BJD_TDB).

## 2. Setting up Your Workspace

### Organizing Your FITS Files
1.  **Create a dedicated folder** for your project (e.g., `MyEclipsingBinary_Project`).
2.  Inside this project folder, create a subfolder named `fits_images` (or similar) and place all your FITS files for the eclipsing binary observation sequence there.
3.  It's also good practice to create folders for `calibration_frames` (darks, flats, biases, if you have them and will calibrate) and `results` (where you'll save your light curves and data tables).

### Launching AstroImageJ
1.  Navigate to the directory where you installed AstroImageJ.
2.  Launch AIJ. This is usually done by double-clicking the `AstroImageJ.jar` file or by running a platform-specific script (e.g., `AstroImageJ.exe` on Windows, or `AIJ_launcher_MacOS.sh` / `AIJ_launcher_Linux.sh`).

### Memory Allocation in AIJ (Important!)
AIJ can be memory-intensive, especially when working with many large FITS files.
1.  Upon launching AIJ, you might see a startup configuration window, or you can access memory settings via **Edit > Options > Memory & Threads...**.
2.  It's recommended to allocate a significant portion of your system's RAM to AIJ, but leave enough for your operating system and other applications. A common recommendation is 50-75% of your available RAM if you have 8GB or more.
3.  For example, if you have 16GB of RAM, you might allocate 8000MB to 12000MB to AIJ.
4.  You'll need to restart AIJ for memory changes to take effect.

## 3. Loading Your FITS Images

1.  In AIJ, go to **File > Open...** to open a single FITS image, or (more likely for a time series) **File > Open Image Series > Open Series (User Selects Folder and Files)...** or **File > Open Image Series > Open All Images in Folder (recursive)...**.
    * For this tutorial, let's assume you'll use **Open Series (User Selects Folder and Files)...**. Navigate to your `fits_images` folder and select all the FITS files for your observation run.
2.  AIJ will load the images as a "stack." The first image in the series will be displayed.
3.  You can navigate through the stack using the slider bar at the bottom of the image window or by using the **Next Slice (`>`)** and **Previous Slice (`<`)** buttons/commands.

### Basic Image Navigation and Display
* **Zoom:** Use the magnifying glass tools or **Image > Zoom > In/Out**.
* **Pan:** Select the hand tool and drag the image.
* **Brightness/Contrast:** Go to **Image > Adjust > Brightness/Contrast... (B&C)**. Click "Auto" for a quick adjustment, or manually adjust the sliders to better see your stars. This only changes the display, not the underlying data values.

## 4. Image Calibration (A Brief Overview)

Raw astronomical images contain instrumental signatures and noise that should ideally be removed for precise photometry. This process is called **image calibration** and typically involves:
* **Bias subtraction:** Removing the baseline electronic signal present even in a zero-second exposure.
* **Dark subtraction:** Removing the signal generated by thermal electrons in the detector, which depends on exposure time and temperature.
* **Flat-fielding:** Correcting for variations in pixel sensitivity across the detector and for artifacts like dust donuts or vignetting.

AIJ has built-in tools for calibration:
1.  Go to **DP** (Data Processor) in the AIJ toolbar (it looks like a little calculator or a graph icon).
2.  In the Data Processor window, click the **Calibrate Images** tab.
3.  Here you can specify your master bias, dark, and flat frames.

**For this tutorial:**
* If you have been provided you with **pre-calibrated FITS files**, you can skip the active calibration steps. However, it's crucial to understand that this step *must* be done for accurate scientific results.
* If you have raw data and calibration frames (bias, darks, flats), you should perform calibration before proceeding with photometry. Consult your mentor or AIJ documentation for detailed steps on creating master calibration frames and applying them.

For now, we will proceed assuming your images are either calibrated or you're learning the photometry steps first with uncalibrated data (which will have more scatter but still show the eclipse).

## 5. Plate Solving and WCS (World Coordinate System)

**Plate solving** is the process of accurately determining the celestial coordinates (Right Ascension - RA, Declination - Dec) for the pixels in your image. This embeds a World Coordinate System (WCS) into the FITS header, which is incredibly useful for:
* Identifying your target and comparison stars by their known coordinates.
* Allowing AIJ to calculate airmass and other astronomically relevant parameters like BJD_TDB.
* Enabling features like image alignment to an external catalog.

**How to Plate Solve in AIJ:**
1.  Make sure your image stack is loaded.
2.  Go to **DP** (Data Processor).
3.  Click the **WCS Plate Solve** tab.
4.  You'll need to provide an initial guess for the image's center coordinates (RA and Dec) and the image scale (arcseconds per pixel). You can often find this information in the FITS header of your images or from your telescope/observatory logs.
    * **RA/Dec Object:** Enter the name of your target (e.g., "V123 Cygni") and click "Get Coords" for AIJ to look it up (requires internet), or manually enter the J2000 RA and Dec.
    * **Image Scale:** Enter your telescope's image scale in arcsec/pixel.
    * **Search Radius:** How far AIJ should search from your initial guess.
5.  Choose a catalog (e.g., "All VizieR Stars" or a specific catalog like Gaia).
6.  Ensure **"Update FITS header with WCS solution"** and **"Update DATE-OBS, TIME-OBS, and exposure time and enable BJD calculation"** are checked for BJD_TDB calculation.
7.  Click **Plate Solve Current Image** or **Plate Solve All Images in Stack**.

**Verifying the Plate Solution:**
* After a successful solve, AIJ will overlay catalog stars on your image. Check if they align well.
* The FITS header will now contain WCS keywords. You can view the header via **Image > Show Info...** (then click FITS Header).

## 6. Performing Aperture Photometry

**Aperture photometry** is the technique of measuring the brightness of a star by summing the pixel values within a circular region (the aperture) centered on the star, and then subtracting the background sky brightness.

1.  Open the **DP (Data Processor)** window.
2.  Go to the **Aperture Photometry** tab.

### Selecting Your Target and Comparison Stars
* **Target Star (T1):** Your eclipsing binary. In the image display, **left-click** on its center.
* **Comparison Stars (C2, C3, ...):** Select 2-5 stable, non-saturated stars of similar brightness if possible. **Left-click** on each.

### Setting Aperture Sizes
Apertures have three radii: Object, Inner Sky, Outer Sky.
1.  **Measure FWHM:** Hover over a star or use AIJ tools to find the FWHM.
2.  In the DP window's photometry settings:
    * **Object Aperture Radius:** ~1.5 to 2.5 x FWHM.
    * **Inner Sky Radius:** Object Radius + (3 to 5 pixels).
    * **Outer Sky Radius:** Inner Sky Radius + (5 to 10 pixels).
3.  Visualize and adjust. Ensure **"Re-center apertures using stellar centroid"** is checked.

## 7. Running the Photometry and Inspecting the Data Table

1.  In the DP window, click **Place Apertures** or **Start Photometry**.
2.  A **"Measurements" table** will be generated.

### Understanding the "Measurements" Table
Key columns:
* **Frame, File_Name**
* **JD_UTC** or **BJD_TDB:** (Barycentric Julian Date - preferred for timing accuracy). Make sure this is calculated; requires accurate FITS header times, observatory coordinates set in AIJ preferences, and WCS solution.
* **AIRMASS**
* **Source-Sky_T1, Source-Sky_C2, ...** (background-subtracted flux)
* **rel_flux_T1:** (Relative flux of T1 - target flux / sum of comparison fluxes). This is crucial!
* **rel_flux_err_T1:** Error in relative flux.

To ensure BJD_TDB is calculated correctly:
* Go to **File > Preferences > Telescope & Site**. Enter your observatory's latitude, longitude, and altitude.
* Ensure FITS headers have correct `DATE-OBS` and `TIME-OBS` (or `UTSTART`).
* WCS plate solving should have been successful.

## 8. Differential Photometry: Creating the Light Curve

Differential photometry uses comparison stars to correct for atmospheric and instrumental effects. The `rel_flux_T1` column already represents this.

**Plotting the Light Curve in AIJ:**
1.  In the Measurements table window, click **Plot**.
2.  In the **"Plot Data"** window:
    * **X-axis:** Choose **BJD_TDB** (or JD_UTC if BJD_TDB is unavailable).
    * **Y-axis:** Choose **rel_flux_T1**.
3.  Click **Update Plot** or **Plot**.

Inspect the initial light curve for eclipses, scatter, and outliers.

## 9. Normalizing the Light Curve

**Normalization** scales the light curve so the out-of-eclipse brightness is typically 1.0.

**Method 1: Using AIJ's Plotting Options**
1.  In the **Plot Data** window, look for a Y-axis option like **"Normalized rel_flux_T1"** or a similar normalization checkbox/setting for the selected Y-axis data. This often normalizes by the median or mean of the series.

**Method 2: Manual Normalization (for more control or if direct option is unclear)**
1.  **Identify Out-of-Eclipse Data:** View your `rel_flux_T1` vs. Time plot. Identify data points representing the system's maximum, combined brightness (out of eclipse).
2.  **Calculate Normalization Factor (`Flux_norm`):**
    * Interactively select these points in the AIJ plot if it provides statistics for selections (mean/median).
    * Or, export the Measurements table (Section 12) and calculate the mean or median of `rel_flux_T1` for these out-of-eclipse points externally.
3.  **Apply Normalization:**
    * **AIJ Custom Column:** If AIJ allows, create a new column (e.g., `norm_flux_T1_manual`) in the Measurements table: `rel_flux_T1 / Flux_norm` (where `Flux_norm` is your calculated value).
    * **External Program:** Perform this division in your spreadsheet/script.
4.  **Re-plot:** Plot your new normalized flux column against Time.

The normalized light curve should have its out-of-eclipse level around 1.0.

## 10. Phase Folding Your Light Curve

**Phase folding** takes your time-series data and wraps it according to a known periodic signal, like the orbit of an eclipsing binary. This overlays all observed cycles, improving the signal-to-noise ratio of the average light curve shape and making it easier to analyze the eclipse profiles.

**Prerequisites for Phase Folding:**
* **Orbital Period (P):** The precise orbital period of your eclipsing binary (e.g., in days). You'll need to get this from astronomical databases (like VSX, GCVS), published literature, or previous analysis of your data (e.g., periodogram analysis).
* **Epoch (T0):** A specific time of reference, usually the time of a primary eclipse minimum (e.g., in BJD_TDB). This sets the zero point for your phase (Phase = 0.0 at primary minimum).
* **Time Data:** Your time measurements, preferably in BJD_TDB, from the Measurements table.
* **Flux Data:** Your (ideally normalized) relative flux data (e.g., `Normalized rel_flux_T1` or `norm_flux_T1_manual`).

**Method 1: Phase Folding Directly in AIJ's Plotting Tool**

AIJ's "Plot Data" or "Multi-Plot Main" window often has built-in capabilities for phase folding:
1.  Open the **Plot Data** window with your Measurements table data.
2.  **Y-axis:** Select your desired flux data (e.g., `Normalized rel_flux_T1` or your manually normalized column).
3.  **X-axis:** Look for an option that says **"Phase"** or similar. If you select this, AIJ will usually enable fields for **Period (P)** and **Epoch (T0 or Zero Phase Epoch)**.
    * Enter your known **Period (P)** value (e.g., `0.78534` days). Make sure the units match your time data (usually days for BJD_TDB).
    * Enter your known **Epoch (T0)** value (e.g., `2458765.12345` BJD_TDB). This should be a time of primary minimum.
4.  Ensure the correct time column (e.g., `BJD_TDB`) is selected or implicitly used by AIJ for the phase calculation if there's such an option.
5.  Click **Update Plot** or **Plot**.

AIJ will calculate the phase for each data point using a formula like: $$Phase = \frac{Time - T0}{P} - \text{floor}\left(\frac{Time - T0}{P}\right)$$
The plot should now show your flux data plotted against orbital phase (usually from 0 to 1, or sometimes -0.5 to 0.5 if a different T0 convention is used).

**Method 2: Calculating Phase Manually (e.g., for external plotting or if AIJ option is not found)**

If you prefer to calculate the phase yourself or if the direct AIJ option is tricky to find/use, you can add a phase column to your Measurements table (either within AIJ if it supports custom column formulas robustly, or after exporting the data):

1.  **Obtain Period (P) and Epoch (T0):** For example, let $P = \text{PERIOD_VALUE}$ days and $T0 = \text{T0_VALUE}$ BJD_TDB.
2.  **Export Data (if not doing in AIJ):** Save your Measurements table (including the BJD_TDB and normalized flux columns) as a CSV file.
3.  **Calculate Phase:** In your preferred software (Python with Pandas, Excel, etc.), create a new column called "Phase". For each row with time $t_i$ (from your BJD_TDB column):
    Calculate the number of cycles: $N_i = \frac{t_i - T0}{P}$
    Calculate the phase: $Phase_i = N_i - \text{floor}(N_i)$
    This ensures $Phase_i$ is always in the range [0, 1).
    *In Python/Pandas, this would be something like: `df['Phase'] = ((df['BJD_TDB'] - T0) / P) % 1.0`*
4.  **Plot:** Plot your normalized flux column against your new "Phase" column.

**Interpreting the Phase-Folded Light Curve:**
* **Phase 0.0 (and 1.0):** Should correspond to the primary eclipse (the deeper one, if T0 was set to a primary minimum).
* **Phase 0.5:** Should correspond to the secondary eclipse (if present and observable).
* The curve should show a well-defined shape, with reduced scatter compared to a single night's observation, as data from multiple orbital cycles are overlaid.
* You can now clearly see the average depth, duration, and shape of the eclipses.

## 11. Refining and Detrending (Optional)

Sometimes, even after differential photometry, your light curve might show residual trends. AIJ offers tools for detrending in the **Plot Data** window under a **"Detrend"** tab/section. You can fit and remove simple functions (linear, polynomial) from out-of-eclipse portions, often using parameters like Airmass.

**Caution:** Detrending should be done carefully. It's generally best to apply detrending to the time-series data *before* phase folding if the trends are related to observing conditions of specific nights rather than an orbital phase-dependent phenomenon.

## 12. Saving Your Results

### Saving the Measurements Table
1.  In the Measurements table window, go to **File > Save Table As...**.
2.  Choose **CSV** or **Tab-delimited Text File (.txt)**. Save it in your `results` folder.
    *If you added a phase column manually outside AIJ, save that modified table.*

### Saving the Light Curve Plot (Time Series and Phase-Folded)
1.  In AIJ's **Plot Data** window:
2.  Use **File > Save Plot As...** or **Right-click > Save As...**.
3.  Save in **PNG** or **PDF** format in your `results` folder. Save both your time-series light curve and your phase-folded light curve.

### Saving AIJ Session/Settings
Look for options like **File > Save Photometry Settings...** in the DP window to save aperture settings etc.

## 13. Conclusion and Next Steps

Congratulations! You've successfully loaded FITS images, performed aperture photometry, created a differential light curve, normalized it, phase-folded it, and saved your results using AstroImageJ.

**Recap of what you learned:**
* The importance of image calibration, WCS, and accurate time (BJD_TDB).
* How to select target and comparison stars for photometry.
* How to set appropriate aperture sizes.
* The meaning of differential photometry and normalization.
* **How to phase fold a light curve using a known period and epoch.**
* The importance of saving your data table and plots.

**Potential Next Steps:**
* **Error Analysis:** Delve deeper into the photometric errors.
* **Modeling the Light Curve:** Use your phase-folded, normalized light curve with software (like PHOEBE, JKTEBOP, or custom scripts) to model and derive physical parameters of the binary system (star radii, inclination, temperatures, etc.).
* **Period Analysis:** If the period wasn't known precisely, you could use tools (e.g., periodograms in AIJ or Python) on your time-series data to determine it.
* **Explore More AIJ Features:** AIJ is packed with many other tools.

**Further Resources:**
* **AstroImageJ User Guide:** The official AIJ website.
* **AstroImageJ Forum/Mailing List.**
